<a href="https://colab.research.google.com/github/arsalanliaquat/SEP788_Project/blob/main/SEP788_ProjectCode-MLP%2BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import time
# pyplot plt
import matplotlib.pyplot as plt
# metrics
from sklearn import metrics
# ignore non-error warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Test Train model selection, cross validation score
from sklearn.model_selection import train_test_split, cross_val_score
# mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
# precision_score, recall_score, confusion_matrix, accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, f1_score, classification_report

# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
# Support Vector Machine SVM
from sklearn.svm import SVC
# Fishers Linear Discriminant FLD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# LogisticRegression
from sklearn.linear_model import LogisticRegression

In [ ]:
# data=np.loadtxt("data_banknote_authentication.txt", delimiter=',')
data = pd.read_table('data_banknote_authentication.txt', delimiter=',', usecols=[0,1,2,3,4], names=['variance', 'skewness', 'curtosis', 'entropy', 'isAuthentic' ], header=None)
data.head()

,variance,skewness,curtosis,entropy,isAuthentic
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [ ]:
print(data.shape)

(1372, 5)


In [ ]:
# Define Feature
Features = ['variance', 'skewness', 'curtosis', 'entropy']
print('Full Features: ', Features)

# Define Target
Target = ['isAuthentic']
print('Target: ', Target)

Full Features:  ['variance', 'skewness', 'curtosis', 'entropy']
Target:  ['isAuthentic']


In [ ]:
# test_data=data[:400]
# train_data=data[400:]
# train=[]
# target=[]

X_train, X_test, y_train, y_test = train_test_split(data[Features], data[Target], test_size = 0.10, random_state = 5)
print(X_train.shape)
print(X_test.shape)

(1234, 4)
(138, 4)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = tf.keras.Sequential()
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
    
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',f1_m])


In [ ]:
start_time = time.time()
model.fit(X_train, y_train, epochs=15)
# model.fit(X_train, y_train)
elapsed_time = time.time() - start_time
print("Training Time  : {:.2}".format(elapsed_time))

Epoch 1/15
39/39 [==============================] - 1s 5ms/step - loss: 0.7848 - accuracy: 0.6135 - f1_m: 0.6827 - precision_m: 0.5353 - recall_m: 0.9761
Epoch 2/15
39/39 [==============================] - 0s 5ms/step - loss: 0.4957 - accuracy: 0.7755 - f1_m: 0.7655 - precision_m: 0.6900 - recall_m: 0.8746
Epoch 3/15
39/39 [==============================] - 0s 4ms/step - loss: 0.3628 - accuracy: 0.8809 - f1_m: 0.8575 - precision_m: 0.8782 - recall_m: 0.8501
Epoch 4/15
39/39 [==============================] - 0s 4ms/step - loss: 0.2895 - accuracy: 0.9222 - f1_m: 0.9020 - precision_m: 0.9541 - recall_m: 0.8621
Epoch 5/15
39/39 [==============================] - 0s 4ms/step - loss: 0.2367 - accuracy: 0.9384 - f1_m: 0.9214 - precision_m: 0.9664 - recall_m: 0.8852
Epoch 6/15
39/39 [==============================] - 0s 5ms/step - loss: 0.1963 - accuracy: 0.9571 - f1_m: 0.9499 - precision_m: 0.9688 - recall_m: 0.9358
Epoch 7/15
39/39 [==============================] - 0s 5ms/step - loss: 0.16

In [ ]:
start_time = time.time()
model.evaluate(X_test,  y_test, verbose=2)
elapsed_time = time.time() - start_time
print("Evaluation Time  : {:.2}".format(elapsed_time))

5/5 - 0s - loss: 0.0610 - accuracy: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - 258ms/epoch - 52ms/step
Evaluation Time  : 0.32


In [ ]:
rnn = tf.keras.Sequential()
rnn.add(keras.Input(shape=(4,1)))
rnn.add(layers.LSTM(16))
rnn.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
# rnn.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])
rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])

In [ ]:
start_time = time.time()
rnn.fit(X_train, y_train, epochs=20)
elapsed_time = time.time() - start_time
print("Training Time  : {:.2}".format(elapsed_time))

Epoch 1/20
39/39 [==============================] - 3s 6ms/step - loss: 0.6332 - accuracy: 0.6694 - f1_m: 0.3822 - precision_m: 1.0000 - recall_m: 0.2442
Epoch 2/20
39/39 [==============================] - 0s 6ms/step - loss: 0.5217 - accuracy: 0.7431 - f1_m: 0.5717 - precision_m: 0.9957 - recall_m: 0.4174
Epoch 3/20
39/39 [==============================] - 0s 6ms/step - loss: 0.3832 - accuracy: 0.9052 - f1_m: 0.8715 - precision_m: 0.9689 - recall_m: 0.8112
Epoch 4/20
39/39 [==============================] - 0s 6ms/step - loss: 0.2624 - accuracy: 0.9595 - f1_m: 0.9508 - precision_m: 0.9554 - recall_m: 0.9497
Epoch 5/20
39/39 [==============================] - 0s 6ms/step - loss: 0.1950 - accuracy: 0.9473 - f1_m: 0.9377 - precision_m: 0.9215 - recall_m: 0.9588
Epoch 6/20
39/39 [==============================] - 0s 6ms/step - loss: 0.1601 - accuracy: 0.9489 - f1_m: 0.9417 - precision_m: 0.9343 - recall_m: 0.9532
Epoch 7/20
39/39 [==============================] - 0s 6ms/step - loss: 0.14

In [ ]:
# test_loss, test_acc = rnn.evaluate(X_test,  y_test, verbose=2)
start_time = time.time()
rnn.evaluate(X_test,  y_test, verbose=2)
elapsed_time = time.time() - start_time
print("Evaluation Time  : {:.2}".format(elapsed_time))

5/5 - 1s - loss: 0.0852 - accuracy: 0.9710 - f1_m: 0.9681 - precision_m: 0.9515 - recall_m: 0.9882 - 659ms/epoch - 132ms/step
Evaluation Time  : 0.72
